In [1]:
# Dependencies
import json
import requests
#from config import api_key
#from citipy import citipy
from random import random
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import datetime
from sqlalchemy import create_engine

In [2]:
# Load Kaggle rainfall csv into data frame
csv_file = "chennai_reservoir_rainfall.csv"
rainfall_df = pd.read_csv(csv_file)
rainfall_df['Date'].replace('/','-',inplace=True)
rainfall_df = rainfall_df[rainfall_df['Date'].str.contains('2019')]
rainfall_df.head()

,Date,POONDI,CHOLAVARAM,REDHILLS,CHEMBARAMBAKKAM
5479,01-01-2019,0.0,0.0,0.0,0.0
5480,02-01-2019,0.0,0.0,0.0,0.0
5481,03-01-2019,0.0,0.0,0.0,0.0
5482,04-01-2019,0.0,0.0,0.0,0.0
5483,05-01-2019,0.0,0.0,0.0,0.0


In [3]:
# Rename column names to keep them short
replace_cols = {'Date':'date', 'POONDI':'pdy','CHOLAVARAM':'clv','REDHILLS':'rdh','CHEMBARAMBAKKAM':'cbr'}
rainfall_df = rainfall_df.rename(columns=replace_cols)
rainfall_df.head()

,date,pdy,clv,rdh,cbr
5479,01-01-2019,0.0,0.0,0.0,0.0
5480,02-01-2019,0.0,0.0,0.0,0.0
5481,03-01-2019,0.0,0.0,0.0,0.0
5482,04-01-2019,0.0,0.0,0.0,0.0
5483,05-01-2019,0.0,0.0,0.0,0.0


In [4]:
# Create postgres connection
rds_connection_string = "postgres:Password00@localhost:5432/etl"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [5]:
# Check postgres connection
engine.table_names()

['customer_name', 'customer_location', 'rainfall', 'weather', 'monthlyagg']

In [6]:
# Load to rainfall table
rainfall_df.to_sql(name='rainfall', con=engine, if_exists='append', index=False)

In [7]:
#Build ~300 records  dataframe. Use random lat & Lon but assume it is for the same city and for each date starting 2019-01-01

Lat = np.random.uniform(low=-90.000, high=90.000, size=300)
Lng = np.random.uniform(low=-180.000, high=180.000, size=300)

LocationDf = pd.DataFrame( 
     list(zip(Lat, Lng) ), columns = ['Lat','Lng'])
LocationDf['Date'] = ''
LocationDf['Max Temp'] = ''


#LocationDf



In [8]:
# Insert dates starting from Jan 1 2019

starttime = datetime.date(2019,1,1)
for index,row in LocationDf.iterrows():
    #LocationDf.loc[index,'Date'] = (starttime + datetime.timedelta(days=index)).strftime("%#d/%#m/%Y")
    LocationDf.loc[index,'Date'] = (starttime + datetime.timedelta(days=index)).strftime("%d-%m-%Y")
    
#LocationDf.groupby(['City','Country'])['City','Country'].count()

#LocationDf.dropna(subset=['City'], axis='rows', inplace=True)
LocationDf.head()


,Lat,Lng,Date,Max Temp
0,-3.901131,51.707714,01-01-2019,
1,-3.629480,2.685189,02-01-2019,
2,-70.791658,-7.276922,03-01-2019,
3,-58.532470,147.102775,04-01-2019,
4,-18.870678,27.117840,05-01-2019,


In [9]:
# Build  query URL to get temperature for each day
baseurl = "http://api.openweathermap.org/data/2.5/weather"
params = {
    'units' : "imperial",
    #'appid' : api_key,
    'appid' : 'c68339f00f348d66865e58c8790c290a',
 #   'city'  : "Chennai"
}



for index,row in LocationDf.iterrows():
    #city = f"{row['City']},{row['Country']}"
    #params['q'] = city
    params['lat'] = row['Lat']
    params['lon'] = row['Lng']
    #print(params)
    response = requests.get(baseurl,params=params).json()
    #print(f"Processing record {i} | {city}")
    print(f"Requesting for {baseurl} with parameters {params}")
    LocationDf.loc[index,'Max Temp'] = response['main']['temp_max']
      
LocationDf.head()



Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -3.9011309050528524, 'lon': 51.707713973915276}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -3.629479999046268, 'lon': 2.685189455407084}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -70.79165842405183, 'lon': -7.276921563905205}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -58.53246968919041, 'lon': 147.1027750060457}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -18.870678033262735, 'lon': 27.11784019122433}
Reques

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 57.28325488042327, 'lon': 71.52176664535827}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -32.918641938721265, 'lon': -108.3767028792337}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -15.429594630224344, 'lon': -142.63595796160314}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -11.965433152167662, 'lon': -102.24168861861646}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 17.679252886406744, 'lon': -13.667625769548039}
R

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -80.86305680386303, 'lon': 85.36395275357279}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 19.041976190585586, 'lon': -81.24281217213573}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -70.03667826929234, 'lon': -4.856204117560537}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -14.883396836624442, 'lon': -87.27625245842722}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -69.42265960721845, 'lon': -134.75425664900996}
Requ

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -79.63889537297804, 'lon': 152.51909277651794}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 72.86827258201492, 'lon': -11.678363788396695}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -14.499256746020151, 'lon': 128.10964485037283}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -83.89586553302078, 'lon': 29.45542478966877}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -14.726328679560297, 'lon': 49.23537338437839}
Reque

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 13.102953566161332, 'lon': 75.61039132989183}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -0.8292431500668727, 'lon': -76.47281152740594}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -65.82927437030528, 'lon': 114.37969562575427}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 25.277989167566446, 'lon': 16.183053387731576}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -72.40994635700974, 'lon': 24.398519348665673}
Reque

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 85.6036423332701, 'lon': -52.018757657966816}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 53.84267805132697, 'lon': 6.78879374123585}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 27.739292722417616, 'lon': -31.184644406817313}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -41.07226986166286, 'lon': -103.58021492726259}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -44.76917484873469, 'lon': -14.219717858341397}
Reques

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 31.85776686728822, 'lon': 138.22803446787606}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 21.79230142177728, 'lon': -91.29487670040531}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 63.95164206456346, 'lon': -31.071341225071222}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -89.02596562433637, 'lon': -8.07243509747812}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -46.874789049677105, 'lon': -97.69876078634695}
Request

Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 59.42922634050859, 'lon': -177.79076209714293}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -88.67319801242435, 'lon': 120.89013982749543}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': -3.536662544744118, 'lon': 77.6348156853839}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 39.23768602581515, 'lon': -136.3025078927291}
Requesting for http://api.openweathermap.org/data/2.5/weather with parameters {'units': 'imperial', 'appid': 'c68339f00f348d66865e58c8790c290a', 'lat': 31.05911350339666, 'lon': 89.35293399414661}


,Lat,Lng,Date,Max Temp
0,-3.901131,51.707714,01-01-2019,76.9
1,-3.629480,2.685189,02-01-2019,72.04
2,-70.791658,-7.276922,03-01-2019,-30.39
3,-58.532470,147.102775,04-01-2019,29.2
4,-18.870678,27.117840,05-01-2019,82.3


In [10]:
# Extract only date and temperature
LocationDf = LocationDf[['Date' , 'Max Temp']]
LocationDf.head()



,Date,Max Temp
0,01-01-2019,76.9
1,02-01-2019,72.04
2,03-01-2019,-30.39
3,04-01-2019,29.2
4,05-01-2019,82.3


In [11]:
#Rename column
LocationDf = LocationDf.rename(columns={'Date':'date','Max Temp': 'temp'})
LocationDf.head()

,date,temp
0,01-01-2019,76.9
1,02-01-2019,72.04
2,03-01-2019,-30.39
3,04-01-2019,29.2
4,05-01-2019,82.3


In [12]:
# Load into weahter table
LocationDf.to_sql(name='weather', con=engine, if_exists='append', index=False)

In [13]:
# Combine the tables, Extract month component and save as separate column
MergedDf = LocationDf.merge(rainfall_df,on='date')

In [14]:
MergedDf['month'] = MergedDf['date'].str[3:]
MergedDf['month']

0      01-2019
1      01-2019
2      01-2019
3      01-2019
4      01-2019
5      01-2019
6      01-2019
7      01-2019
8      01-2019
9      01-2019
10     01-2019
11     01-2019
12     01-2019
13     01-2019
14     01-2019
15     01-2019
16     01-2019
17     01-2019
18     01-2019
19     01-2019
20     01-2019
21     01-2019
22     01-2019
23     01-2019
24     01-2019
25     01-2019
26     01-2019
27     01-2019
28     01-2019
29     01-2019
        ...   
138    05-2019
139    05-2019
140    05-2019
141    05-2019
142    05-2019
143    05-2019
144    05-2019
145    05-2019
146    05-2019
147    05-2019
148    05-2019
149    05-2019
150    05-2019
151    06-2019
152    06-2019
153    06-2019
154    06-2019
155    06-2019
156    06-2019
157    06-2019
158    06-2019
159    06-2019
160    06-2019
161    06-2019
162    06-2019
163    06-2019
164    06-2019
165    06-2019
166    06-2019
167    06-2019
Name: month, Length: 168, dtype: object

In [15]:
# create total or rainfall columns and average for temperature and save as data frame
rdhsum = MergedDf.groupby(MergedDf['month'])['rdh'].sum()
pdysum = MergedDf.groupby(MergedDf['month'])['pdy'].sum()
clvsum = MergedDf.groupby(MergedDf['month'])['clv'].sum()
cbrsum = MergedDf.groupby(MergedDf['month'])['cbr'].sum()
tempavg = MergedDf.groupby(MergedDf['month'])['temp'].sum()/MergedDf.groupby(MergedDf['month'])['temp'].count()

FinalDf = pd.DataFrame({ 'rdhsum' : rdhsum, 'pdysum' : pdysum , 'clvsum':clvsum,'cbrsum':cbrsum,'tempavg':tempavg})
FinalDf.reset_index(inplace=True)
FinalDf



,month,rdhsum,pdysum,clvsum,cbrsum,tempavg
0,01-2019,0.0,0.0,0.0,0.0,36.800323
1,02-2019,0.0,4.0,0.0,0.0,54.672143
2,03-2019,0.0,0.0,0.0,0.0,41.644516
3,04-2019,1.0,6.0,3.0,0.0,45.672667
4,05-2019,0.0,27.0,0.0,0.0,29.383548
5,06-2019,0.0,0.0,0.0,0.0,40.142353


In [16]:
# Save data frame into monthlyagg table
FinalDf.to_sql(name='monthlyagg', con=engine, if_exists='append', index=False)

In [17]:
#alternate method
outputDf = pd.read_sql(con=engine, sql='select substring(r.date, 4,7) as month, sum(rdh), sum(pdy), sum(clv), sum(cbr),avg(temp) from rainfall r join weather w on r.date = w.date group by month order by month')
outputDf

,month,sum,sum,sum,sum,avg
0,01-2019,0.0,0.0,0.0,0.0,36.800323
1,02-2019,0.0,4.0,0.0,0.0,54.672143
2,03-2019,0.0,0.0,0.0,0.0,41.644516
3,04-2019,1.0,6.0,3.0,0.0,45.672667
4,05-2019,0.0,27.0,0.0,0.0,29.383548
5,06-2019,0.0,0.0,0.0,0.0,40.142353
